In [17]:
!pip install git+https://github.com/shahashka/SP-GIES

  Cloning https://github.com/shahashka/SP-GIES to /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-s9o7wc43
  Running command git clone --filter=blob:none --quiet https://github.com/shahashka/SP-GIES /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-s9o7wc43
  Resolved https://github.com/shahashka/SP-GIES to commit 07e798636a8759b3ca38f05f253a1dd2983b014c
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... - done


In [18]:
from SP-GIES import utils

SyntaxError: invalid syntax (1057630954.py, line 1)